OpenVINO 준비

In [1]:
%pip install -q "openvino>=2023.1.0"

In [2]:
!pip install openvino-dev

keras .tf 모델을 Intel OpenVINO IR 파일(xml, bin)로 한다.

In [3]:
import openvino
import openvino as ov
from openvino.tools.mo import convert_model

BASE_PATH = './drive/MyDrive/Colab Notebooks/data/'
ov_model = convert_model(BASE_PATH+'bin/mobilenetv2_class4.tf')
ov.save_model(ov_model, BASE_PATH+'bin/mobilenetv2_class4.xml')

### 테스트 파일 목록으로 읽어와서 답안 출력하기.



In [4]:
# --------------------------- Step 1. Initialize OpenVINO Runtime Core ------------------------------------------------
core = ov.Core()

In [5]:
# --------------------------- Step 2. Read a model --------------------------------------------------------------------
# (.xml and .bin files) or (.onnx file)
model = core.read_model(BASE_PATH+'bin')

if len(model.inputs) != 1: print('Sample supports only single input topologies')
if len(model.outputs) != 1: print('Sample supports only single output topologies')

In [6]:
import os

directory_path = "./drive/MyDrive/Colab Notebooks/data/test/MildDemented"
#directory_path = "./drive/MyDrive/Colab Notebooks/data/test"

# Get a list of all files in the directory
file_list = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Print the list of files
print("List of files in the directory:")
for file in file_list:
    print(file)

List of files in the directory:
26 (23).jpg
27 (15).jpg
27 (23).jpg
26 (19).jpg
26 (20).jpg
27 (19).jpg
26 (24).jpg
26 (28).jpg
27 (17).jpg
26 (21).jpg
26 (26).jpg
27 (18).jpg
27 (22).jpg
27 (24).jpg
27 (26).jpg
26 (25).jpg
26.jpg
27 (14).jpg
27 (12).jpg
27 (13).jpg
27 (21).jpg
27 (25).jpg
27 (20).jpg
27 (10).jpg
27 (16).jpg
27 (11).jpg
26 (22).jpg
27 (2).jpg
26 (27).jpg
28 (16).jpg
30 (14).jpg
28 (8).jpg
28 (19).jpg
29 (17).jpg
30 (13).jpg
28 (17).jpg
29 (7).jpg
29 (6).jpg
29 (11).jpg
29 (21).jpg
28 (12).jpg
28 (9).jpg
29 (15).jpg
27 (9).jpg
29 (10).jpg
27 (3).jpg
28 (25).jpg
29 (22).jpg
28 (7).jpg
28 (13).jpg
28 (3).jpg
28.jpg
28 (18).jpg
27 (6).jpg
27 (4).jpg
28 (27).jpg
27 (28).jpg
28 (4).jpg
28 (5).jpg
28 (22).jpg
29 (4).jpg
29 (13).jpg
29.jpg
29 (5).jpg
29 (8).jpg
28 (6).jpg
29 (2).jpg
28 (26).jpg
28 (10).jpg
30 (11).jpg
28 (15).jpg
29 (25).jpg
29 (3).jpg
29 (27).jpg
29 (23).jpg
30 (12).jpg
28 (21).jpg
29 (26).jpg
29 (14).jpg
28 (11).jpg
29 (24).jpg
28 (24).jpg
27.jpg
29 (28).jpg

In [7]:
import cv2
import numpy as np

# --------------------------- Step 3. Apply preprocessing -------------------------------------------------------------
ppp = ov.preprocess.PrePostProcessor(model)

h = 224
w = 224

# 1) Set input tensor information:
# - input() provides information about a single model input
# - reuse precision and shape from already available `input_tensor`
# - layout of data is 'NHWC'
ppp.input().tensor() \
    .set_shape((1,3,224,224)) \
    .set_element_type(ov.Type.u8) \
    .set_layout(ov.Layout('NHWC'))  # noqa: ECE001, N400

# 2) Adding explicit preprocessing steps:
# - apply linear resize from tensor spatial dims to model spatial dims
ppp.input().preprocess().resize(ov.preprocess.ResizeAlgorithm.RESIZE_LINEAR)

# 3) Here we suppose model has 'NCHW' layout for input
ppp.input().model().set_layout(ov.Layout('NCHW'))

# 4) Set output tensor information:
# - precision of tensor is supposed to be 'f32'
ppp.output().tensor().set_element_type(ov.Type.f32)

# 5) Apply preprocessing modifying the original 'model'
model = ppp.build()

파일 추론하기

In [8]:
import pandas as pd
import cv2
import numpy as np

# --------------------------- Step 4. Loading model to the device -----------------------------------------------------
print('Loading the model to the plugin')
compiled_model = core.compile_model(model, 'CPU')
img_height = 224

result = []

res = open(BASE_PATH+"result.txt", "w")

print('Starting inference in synchronous mode')

for file in file_list:
    # --------------------------- Step 5. Set up input --------------------------------------------------------------------
    # Read input image
    image_path = directory_path + "/" + file
    org_image = cv2.imread(image_path)
    image = cv2.resize(org_image, dsize=(img_height,img_height), interpolation=cv2.INTER_LINEAR)
    # Add N dimension
    nchw_tensor = np.expand_dims(image, 0)

    # NHWC -> NCHW
    input_tensor = np.transpose(nchw_tensor, (0,3,1,2))

    # --------------------------- Step 6. Create infer request and do inference synchronously -----------------------------
    results = compiled_model.infer_new_request({0:input_tensor})

    # --------------------------- Step 7. Process output ------------------------------------------------------------------
    predictions = next(iter(results.values()))

    # Change a shape of a numpy.ndarray with results to get another one with one dimension
    probs = predictions.reshape(-1)

    # Get an array of 4 class IDs in descending order of probability
    top_4 = np.argsort(probs)[-4:][::-1]

    header = 'class_id probability'
    print(f'Image path: {image_path}')
    print('Top 4 results: ')
    print(header)
    print('-' * len(header))

    for class_id in top_4:
        probability_indent = ' ' * (len('class_id') - len(str(class_id)) + 1)
        print(f'{class_id}{probability_indent}{probs[class_id]:.7f}')

    print(top_4[0])
    result.append(top_4[0])
    res.write(file+':'+str(top_4[0])+'\n')

res.write(str(result)+'\n')
res.close()

Loading the model to the plugin
Starting inference in synchronous mode
Image path: ./drive/MyDrive/Colab Notebooks/data/test/MildDemented/26 (23).jpg
Top 4 results: 
class_id probability
--------------------
2        0.9688463
3        0.0259657
0        0.0036490
1        0.0015391
2
Image path: ./drive/MyDrive/Colab Notebooks/data/test/MildDemented/27 (15).jpg
Top 4 results: 
class_id probability
--------------------
2        0.8963237
3        0.0535133
0        0.0365097
1        0.0136534
2
Image path: ./drive/MyDrive/Colab Notebooks/data/test/MildDemented/27 (23).jpg
Top 4 results: 
class_id probability
--------------------
2        0.7906199
3        0.1318483
0        0.0556009
1        0.0219309
2
Image path: ./drive/MyDrive/Colab Notebooks/data/test/MildDemented/26 (19).jpg
Top 4 results: 
class_id probability
--------------------
2        0.7555459
3        0.1252284
0        0.0603969
1        0.0588287
2
Image path: ./drive/MyDrive/Colab Notebooks/data/test/MildDemented/26